## Setup

In [1]:
# Import required modules.
import os
import cv2
import numpy as np
import tensorflow.compat.v1 as tf1
import tensorflow as tf2
from pathlib import Path
from tqdm import tqdm
from pydnet.data import KITTI
from pydnet.models import Pydnet


# Path to KITTI dataset.
KITTI_PATH = Path("../")\
    / "data"\
    / "mount"\
    / "KITTI"\
    / "raw_data"

# Path to dataset slice index.
SLICE_PATH = Path("../")\
    / "data"\
    / "slices"\
    / "test_files.txt"

# Path to PyDnet pretrained checkpoint.
CHECK_PATH = Path("../")\
    / "data"\
    / "checkpoint"\
    / "pydnet"\
    / "pydnet"


# Disable Tensorflow warning messages.
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2" 

# Run Tensorflow in earger mode.
if tf1.executing_eagerly():
   tf1.disable_eager_execution()


# Utility functions.
def read_lines(path: Path) -> list:
    """Read test files from path.
    """
    assert path.exists()
    with open(path, "r") as f:
        lines = f.readlines()
    return [l.strip() for l in lines]

## Load KITTI and PydNet

In [2]:
# Load KITTI dataset.
dataset = KITTI({
    "h": 320,
    "w": 640,
    "path": KITTI_PATH,
    "slice": SLICE_PATH
})

# Build PydNet.
network = Pydnet({
    "h": 320,
    "w": 640,
    "is_training": False
})

## Start Tensorflow Session and Run Inference

In [3]:
# Setup KITTI dataset feeding placeholder.
pred = network.forward(dataset.batch)
pred = tf1.nn.relu(pred)

# Setup Tensorflow session and restore checkpoint.
save = tf1.train.Saver()
sess = tf1.Session()
sess.run(tf1.global_variables_initializer())
sess.run(dataset.initializer)
save.restore(sess, str(CHECK_PATH))

INFO:tensorflow:Restoring parameters from ../data/checkpoint/pydnet/pydnet
